# Assignment 4:  Recommendation systems

Here we'll implement a content-based recommendation algorithm.

In [9]:
import math
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
movie_names = ['Avatar', 'Get Shorty', 'Princess Bride', 'Goonies']
movie_descriptions = [
    'an epic science fiction film',
    'a crime thriller comedy film; adapted from a book',
    'a fantasy comedy adventure film; adapted from a book',
    'an adventure comedy film'
]
# ratings: row=user, col=movie
# All ratings are between 0 and 1, with 1 meaning the user really liked the movie.
# A 0 value means the user has not rated the movie.
ratings = csr_matrix([
    [.1, 0, .2, 0],
    [0, .9, 0, .3],
    [.3, 0, .9, 0],
    [0, 0, 0, .4],
    [0, 0, .3, .4]
    ])

In [11]:
# Do not modify.
vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b', dtype=np.float)
movie_term_matrix = vectorizer.fit_transform(movie_descriptions)
vocabulary = np.array(vectorizer.get_feature_names())
print('movie_term_matrix:')
print(movie_term_matrix.todense())
print('vocabulary')
print(vocabulary)
# movie_term_matrix: value i,j is the frequency of term j in the description of movie i.

movie_term_matrix:
[[ 0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.]
 [ 2.  1.  0.  0.  1.  1.  1.  0.  0.  0.  1.  1.  0.  1.]
 [ 2.  1.  1.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.]
 [ 0.  0.  1.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.]]
vocabulary
[u'a' u'adapted' u'adventure' u'an' u'book' u'comedy' u'crime' u'epic'
 u'fantasy' u'fiction' u'film' u'from' u'science' u'thriller']


In [12]:
def document_frequencies(movie_term_matrix):
    """ Compute the number of different documents that each term appears in.
    Params::
      movie_term_matrix...csr_matrix where entry i,j is the number
                          of times term j appears in document i
    Returns:
      a numpy array with one element per term in the vocabulary."""
    ###TODO
    ###
    dfs = []
    for i in range(0,movie_term_matrix.todense().shape[1]):
        currentTermDfs = 0.0
        for j in range(0,movie_term_matrix.todense().shape[0]):            
            if movie_term_matrix[j,i] > 0:
                currentTermDfs = currentTermDfs + 1.0               
        dfs.append(currentTermDfs)
    
    return np.array(dfs)
    
    
    
dfs = document_frequencies(movie_term_matrix)
dfs

array([ 2.,  2.,  2.,  2.,  2.,  3.,  1.,  1.,  1.,  1.,  4.,  2.,  1.,  1.])

In [13]:
def tfidf(movie_term_matrix, dfs):
    """ Create a new matrix that transforms movie_term_matrix using tfidf.
    Simply divide each value by the document frequency for that term.
    
    Params:
      movie_term_matrix...csr_matrix where entry i,j is the number
                          of times term j appears in document i
      dfs.................document frequencies for each term.
    Returns:
      A csr_matrix that is a copy of movie_term_matrix where value
      i,j is divided by the document frequency of term j"""
    ###TODO    
    ###
    for row in range(0,movie_term_matrix.todense().shape[0]):
        for col in range(0,movie_term_matrix.todense().shape[1]):
            movie_term_matrix[row,col] = movie_term_matrix[row,col] / dfs[col]
            
    return movie_term_matrix
    
# tfidf matrix: row=movie, col=term
tfidf_matrix = tfidf(movie_term_matrix, dfs)
tfidf_matrix.todense()

matrix([[ 0.        ,  0.        ,  0.        ,  0.5       ,  0.        ,
          0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
          0.25      ,  0.        ,  1.        ,  0.        ],
        [ 1.        ,  0.5       ,  0.        ,  0.        ,  0.5       ,
          0.33333333,  1.        ,  0.        ,  0.        ,  0.        ,
          0.25      ,  0.5       ,  0.        ,  1.        ],
        [ 1.        ,  0.5       ,  0.5       ,  0.        ,  0.5       ,
          0.33333333,  0.        ,  0.        ,  1.        ,  0.        ,
          0.25      ,  0.5       ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.5       ,  0.5       ,  0.        ,
          0.33333333,  0.        ,  0.        ,  0.        ,  0.        ,
          0.25      ,  0.        ,  0.        ,  0.        ]])

In [14]:
def make_user_profiles(ratings, tfidf_matrix):
    """
    Create a user profile matrix by computing the weighted average of the tfidf
    vectors of each movie he has rated. E.g., if a person has rated 
    one movie .2 with tfidf vector ([.1, .3]) and rated another movie
    .6 with tfidf vector([.2, .4]), then the weighted average is:
    [(.2*.1 + .6*.2) / (.2 + .6), (.2*.3 + .6*.4) / (.2 + .6)]
    Params:
      ratings........the user/movie ratings matrix
      tfidf_matrix...the movie/term tfidf matrix
    Returns:
      A csr matrix where each row represents a user and the columns represent terms.
    """
    ###TODO
    ###
    userProfileMatrix = []
    
    for userRatings in ratings.todense():
        w_avgList = []
        for i in range(0,tfidf_matrix.todense().shape[1]): #no of columns
            
            termColumn = tfidf_matrix.todense()[:,i]
            w_avg = 0.0
            sum = 0.0        
                                    
            for j in range(0,len(termColumn)):                                
                sum = sum + (userRatings[0,j] * termColumn[j,0])
            
            matrixSum = 0.0
            for j in range(0,len(termColumn)):
                matrixSum = matrixSum + userRatings[0,j] 
                         
            w_avg = sum / matrixSum                        
            w_avgList.append(w_avg)
                        
        userProfileMatrix.append(w_avgList)
    
    return csr_matrix(userProfileMatrix)
    
        

user_profiles = make_user_profiles(ratings, tfidf_matrix)
user_profiles.todense()

matrix([[ 0.66666667,  0.33333333,  0.33333333,  0.16666667,  0.33333333,
          0.22222222,  0.        ,  0.33333333,  0.66666667,  0.33333333,
          0.25      ,  0.33333333,  0.33333333,  0.        ],
        [ 0.75      ,  0.375     ,  0.125     ,  0.125     ,  0.375     ,
          0.33333333,  0.75      ,  0.        ,  0.        ,  0.        ,
          0.25      ,  0.375     ,  0.        ,  0.75      ],
        [ 0.75      ,  0.375     ,  0.375     ,  0.125     ,  0.375     ,
          0.25      ,  0.        ,  0.25      ,  0.75      ,  0.25      ,
          0.25      ,  0.375     ,  0.25      ,  0.        ],
        [ 0.        ,  0.        ,  0.5       ,  0.5       ,  0.        ,
          0.33333333,  0.        ,  0.        ,  0.        ,  0.        ,
          0.25      ,  0.        ,  0.        ,  0.        ],
        [ 0.42857143,  0.21428571,  0.5       ,  0.28571429,  0.21428571,
          0.33333333,  0.        ,  0.        ,  0.42857143,  0.        ,
          0.

Next, we'll predict a user's rating based on the cosine similarity between the user's profile and the item's tfidf vector.

In [15]:
def norm(vector):
    """
    Compute the Euclidean norm of one row of a csr_matrix.
    https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm
    Input:
      vector...one row from a csr_matrix
    Returns:
      a float, the Euclidean norm of the vector.
    """
    ###TODO
    ###    
    
    rms = 0.0
    
    for i in range(0,vector.shape[1]):
         rms = rms + (vector.todense()[0,i]*vector.todense()[0,i])    
    return math.sqrt(rms)

norm(csr_matrix([3,4]))

5.0

In [16]:
def cosine(v1, v2):
    """
    Compute the cosine similarity between two vectors (rows from a csr_matrix).
    https://en.wikipedia.org/wiki/Cosine_similarity
    Params:
      v1...one vector
      v2...another vector
    Returns:
      a float representing the cosine similarity/
    """
    ###TODO
    ###
    numerator = 0.0
    denominator1 = 0.0
    denominator2 = 0.0
    
    for i in range(0,v1.shape[1]):
        numerator = numerator + (v1.todense()[0,i]*v2.todense()[0,i])
        denominator1 = denominator1 + (v1.todense()[0,i]*v1.todense()[0,i])
        denominator2 = denominator2 + (v2.todense()[0,i]*v2.todense()[0,i])
    
    return (numerator/((math.sqrt(denominator1) * (math.sqrt(denominator2)))))
                       
round(cosine(csr_matrix([2,4]), csr_matrix([3,8])), 5)

0.99451

In [17]:
def predict_ratings_w_user_profiles(ratings, user_profiles, tfidf_matrix):
    """
    Make a copy of the ratings matrix. Replace each 0 entry with a predicted score
    based on user_profile. Specifically, the ratings of user i for movie j is the 
    cosine similarity between user i's profile and movie's j tfidf vector.
    
    Params:
      ratings.........the user x movie ratings matrix.
      user_profiles...the user x term profile matrix
      tfidf_matrix....the move x term tfidf matrix
    Returns:
      a user x movie csr_matrix of ratings. It should be a copy of the original
      ratings matrix, where 0 values have been replaced by the prediced rating.
    """
    ###TODO
    ###    
    ratingsCopy = ratings
    for userId,userRatings in enumerate(ratingsCopy.todense()):  
        userRatings = np.array(userRatings)[0].tolist()
        for movieId,movieRating in enumerate(userRatings):
            if movieRating == 0.0:
                ratingsCopy[userId,movieId] = cosine(csr_matrix(np.array(user_profiles.todense())[userId].tolist()),csr_matrix(np.array(tfidf_matrix.todense())[movieId].tolist())                    )
    
    return ratingsCopy

predicted = predict_ratings_w_user_profiles(ratings, user_profiles, tfidf_matrix)
predicted.todense()

matrix([[ 0.1       ,  0.48953468,  0.2       ,  0.35045309],
        [ 0.04514693,  0.9       ,  0.57142822,  0.3       ],
        [ 0.3       ,  0.51857187,  0.9       ,  0.33970573],
        [ 0.20920278,  0.10678984,  0.28972493,  0.4       ],
        [ 0.11225271,  0.46388588,  0.3       ,  0.4       ]])

In [18]:
import json

def doround(x):
    return ['%.3f' % i for i in x]

outf = open('output.txt', 'wt')
json.dump(
    {
        'dfs': doround(document_frequencies(movie_term_matrix).tolist()),
        'tfidf': doround(tfidf(movie_term_matrix, dfs).todense().A1),
        'profiles': doround(make_user_profiles(ratings, tfidf(movie_term_matrix, dfs)).todense().A1),
        'norm': round(norm(csr_matrix([5,6])), 2),
        'cosine': round(cosine(csr_matrix([5,6]), csr_matrix([-5,-1])), 2),
        'predicted': doround(predict_ratings_w_user_profiles(ratings,
                                                     make_user_profiles(ratings, tfidf(movie_term_matrix, dfs)),
                                                     tfidf(movie_term_matrix, dfs)).todense().A1),
    },
    outf, indent=2, sort_keys=True)
outf.close()
    